In [ ]:
# Imports
import numpy as np
from radiotools import plthelpers as php
from matplotlib import pyplot as plt

import os
import time
from radiotools import helper as hp
from NuRadioReco.utilities import units
import pickle
import argparse
from termcolor import colored
from toolbox import load_file, models_dir, calculate_percentage_interval
from constants import datapath, data_filename, label_filename, test_file_ids, plots_dir
# -------

In [ ]:
#GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using",device,". GPU # is",torch.cuda.current_device())

# # Parse arguments
# parser = argparse.ArgumentParser(description='Evaluate energy resolution')
# parser.add_argument("run_id", type=str ,help="the id of the run, eg '3.2' for run3.2")

# args = parser.parse_args()
# run_id = args.run_id

run_id = "Model.8"
# Save the run name and filename
run_name = f"run{run_id}"
# filename = f"model_history_log_{run_name}.csv"

# Models folder
saved_model_dir = models_dir(run_name)
print(saved_model_dir)
print(colored(f"plotting samples for {run_name}...", "yellow"))

# Load the model
from generator import E_Model
import pytorch_lightning as pl
import jammy_flows

# mymodel = E_Model().to(device)
mymodel = E_Model()
save_model_path=os.path.join(saved_model_dir,  f"{run_name}.pt")
mymodel.load_state_dict(torch.load(save_model_path))
mymodel.eval()
mymodel.double()

# Load test file data and make predictions
# way 1
from generator import Prepare_Dataset
from tqdm import tqdm
from torch.utils.data import DataLoader
from constants import test_data_points, sample_numbers

list_of_file_ids_test_small = np.random.choice(test_file_ids, size=1, replace=False)
test = Prepare_Dataset(file_ids=list_of_file_ids_test_small, points = 9)
print("Picked test set ids:",list_of_file_ids_test_small)
print("Length of test dataset: ", len(test))

test_loader = DataLoader(test, batch_size=1, shuffle=False)


In [ ]:
plots_dir = f"{plots_dir}/{run_id}/samples"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

In [ ]:
from scipy.optimize import curve_fit
def gauss_fit_plot(sample_data, energysmallbin=''):   
    (count, bins) = np.histogram(sample_data, bins=200)#np.linspace(-40, 40, 400)
                                 #bins=np.linspace(np.min(sigma_ratio_data)-0.1, np.max(sigma_ratio_data)+0.1, 100))

    bins_middle = (bins[:-1] + bins[1:])/2
    p0 = [200, 18., 0.5]
    def gauss(x, *p):
        A, mu, sigma = p
        return A*(2 * np.pi * sigma)**-0.5 * np.exp(-0.5 * (x - mu) ** 2 * sigma ** -2)
    coeff, var_matrix = curve_fit(gauss, bins_middle, count, p0=p0, maxfev = 5000)
    hist_fit = gauss(bins_middle, *coeff)

    fig, ax = php.get_histogram(sample_data, bins=200, xlabel="log(E)")
                                #bins=np.linspace(np.min(sigma_ratio_data)-0.1, np.max(sigma_ratio_data)+0.1, 100),

    ax.plot(bins_middle, hist_fit)
    ax.legend([f"Gaussian fit with \n$\mu$ = {coeff[1]:.2f}\n$\sigma$ = {coeff[2]:.2f}"], loc='upper right')
    plt.title(f"Predicted sample distribution for {run_name} with\n68 % interval of E at {energy_68:.2f}, ture E = {y.item():.2f}")
    fig.savefig(f"{plots_dir}/Predicted_sample_trueE_{y.item():.2f}_{run_name}_.png")

In [ ]:
with torch.no_grad():
    # Iterate through test set minibatchs 
    for x, y in tqdm(test_loader):
        target_sample_list = []
        conv_out = mymodel.forward(x)
        target_sample, base_sample, target_log_pdf, base_log_pdf = mymodel.pdf.sample(samplesize=sample_numbers, conditional_input=conv_out.repeat(sample_numbers,1))
        
        sample_dist = np.squeeze(target_sample.numpy())
        energy_68 = calculate_percentage_interval(sample_dist, 0.68)
        gauss_fit_plot(sample_dist)
        